In [ ]:

!pip install -qqq -U openai langchain wandb

In [ ]:
import datetime

from langchain.chains import LLMChain
from langchain.llms.fake import FakeListLLM
from langchain.prompts import PromptTemplate

import wandb
from wandb_addons.prompts import Trace

In [ ]:
PROJECT="high_level_trace"

In [ ]:
#trace langchain chains
run = wandb.init(project=PROJECT)
llm = FakeListLLM(responses=[f"Fake response: {i}" for i in range(100)])
prompt_template = "What is a good name for a company that makes {product}?"
prompt = PromptTemplate(
    input_variables=["product"],
    template=prompt_template,
)

chain = LLMChain(llm=llm, prompt=prompt)

for i in range(2):
    product = f"q: {i} - {datetime.datetime.now().timestamp()}"
    start_time_ms = datetime.datetime.now().timestamp() * 1000
    response = chain(product)
    end_time_ms = datetime.datetime.now().timestamp() * 1000
    trace = Trace(name=f"fake_chain_{i}",
          kind="chain",
          status_code="success",
          metadata=None,
          start_time_ms=start_time_ms,
          end_time_ms=end_time_ms,
          inputs={"prompt":prompt_template.format(product=product)},
          outputs={"response": response["text"]},
          )
    trace.log(name=f"trace_{i}")
run.finish()

In [ ]:

from getpass import getpass
import openai

openai.api_key = getpass("Please enter your openai api key")

In [ ]:
# trace openai api calls
run = wandb.init(project=PROJECT)
request_kwargs = dict(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {
            "role": "assistant",
            "content": "The Los Angeles Dodgers won the World Series in 2020.",
        },
        {"role": "user", "content": "Where was it played?"},
    ],
)

start_time_ms = datetime.datetime.now().timestamp() * 1000
response = openai.ChatCompletion.create(**request_kwargs)
end_time_ms = datetime.datetime.now().timestamp() * 1000

trace = Trace(
    name=f"openai_chat_completion",
    kind="llm",
    status_code="success",
    metadata={"model": "gpt-3.5-turbo"},
    start_time_ms=start_time_ms,
    end_time_ms=end_time_ms,
    inputs={"messages":request_kwargs["messages"]},
    outputs={"response": response.choices[0]["message"]["content"]},
)

trace.log(name=f"openai_trace")
run.finish()